In [1]:
import numpy as np
import pandas as pd

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import mlflow
from mlflow.models import infer_signature

In [68]:
import pandas as pd
import numpy as np

# Load the data with your previous code
df = pd.read_csv(r"C:\Users\admin\Documents\Projects\MLE\data\bank.csv", sep=";")
df = df.drop(columns=["day", "month", "duration", "pdays", "campaign"])
df = df.reset_index().rename(columns={"index": "CIF"})

# Convert customer_age to numeric
df['age'] = pd.to_numeric(df['age'])

# Generate random tenure values that are less than age
# We'll use a logical approach where:
# 1. Younger customers (18-25) typically have shorter tenure (0-5 years)
# 2. Middle-aged customers (26-50) can have medium tenure (0-15 years)
# 3. Older customers (51+) can have longer tenure (0-30 years, but still less than age)

def generate_tenure(age):
    if age <= 25:
        # Younger customers: max tenure of 5 years or 25% of age, whichever is less
        max_tenure = min(5, age * 0.25)
        return max(0, round(np.random.uniform(0, max_tenure), 1))
    elif age <= 50:
        # Middle-aged: max tenure of 15 years or 40% of age
        max_tenure = min(15, age * 0.4)
        return max(0, round(np.random.uniform(0, max_tenure), 1))
    else:
        # Older customers: max tenure of 30 years or 60% of age
        max_tenure = min(30, age * 0.6)
        return max(0, round(np.random.uniform(0, max_tenure), 1))

# Apply the function to generate tenure values
np.random.seed(42)  # For reproducibility
df['tenure'] = df['age'].apply(generate_tenure)

# Rename columns to more comprehensive names
column_renaming = {
    "age": "customer_age",
    "job": "employment_type",
    "marital": "marital_status",
    "education": "education_level",
    "default": "credit_default",
    "balance": "account_balance",
    "housing": "housing_loan",
    "loan": "personal_loan",
    "contact": "contact_method",
    "previous": "previous_campaign_number_of_contacts",
    "poutcome": "previous_campaign_outcome",
}
df = df.rename(columns=column_renaming)


In [69]:
date_value = pd.to_datetime('2024-03-31')
df['date'] = date_value

In [70]:
df

,CIF,customer_age,employment_type,marital_status,education_level,credit_default,account_balance,housing_loan,personal_loan,contact_method,previous_campaign_number_of_contacts,previous_campaign_outcome,y,tenure,date
0,0,30,unemployed,married,primary,no,1787,no,no,cellular,0,unknown,no,4.5,2024-03-31
1,1,33,services,married,secondary,no,4789,yes,yes,cellular,4,failure,no,12.5,2024-03-31
2,2,35,management,single,tertiary,no,1350,yes,no,cellular,1,failure,no,10.2,2024-03-31
3,3,30,management,married,tertiary,no,1476,yes,yes,unknown,0,unknown,no,7.2,2024-03-31
4,4,59,blue-collar,married,secondary,no,0,yes,no,unknown,0,unknown,no,4.7,2024-03-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4516,4516,33,services,married,secondary,no,-333,yes,no,cellular,0,unknown,no,6.9,2024-03-31
4517,4517,57,self-employed,married,tertiary,yes,-3313,yes,yes,unknown,0,unknown,no,26.9,2024-03-31
4518,4518,57,technician,married,secondary,no,295,no,no,cellular,0,unknown,no,5.9,2024-03-31
4519,4519,28,blue-collar,married,secondary,no,1137,no,no,cellular,3,other,no,0.3,2024-03-31


In [71]:
df = df[['CIF', 'date', 'customer_age', 'tenure', 'employment_type', 'marital_status',
       'education_level', 'credit_default', 'account_balance', 'housing_loan',
       'personal_loan', 'contact_method', 'previous_campaign_number_of_contacts',
       'previous_campaign_outcome', 'y']]
df['y'] = (df['y'] == 'yes').astype(int)

In [73]:
df.to_excel(r"C:\Users\admin\Documents\Projects\MLE\data\bank.xlsx", index= False)

In [74]:
(
    pd.read_excel(r"C:\Users\admin\Documents\Projects\MLE\data\bank.xlsx")
)

,CIF,date,customer_age,tenure,employment_type,marital_status,education_level,credit_default,account_balance,housing_loan,personal_loan,contact_method,previous_campaign_number_of_contacts,previous_campaign_outcome,y
0,0,2024-03-31,30,4.5,unemployed,married,primary,no,1787,no,no,cellular,0,unknown,0
1,1,2024-03-31,33,12.5,services,married,secondary,no,4789,yes,yes,cellular,4,failure,0
2,2,2024-03-31,35,10.2,management,single,tertiary,no,1350,yes,no,cellular,1,failure,0
3,3,2024-03-31,30,7.2,management,married,tertiary,no,1476,yes,yes,unknown,0,unknown,0
4,4,2024-03-31,59,4.7,blue-collar,married,secondary,no,0,yes,no,unknown,0,unknown,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4516,4516,2024-03-31,33,6.9,services,married,secondary,no,-333,yes,no,cellular,0,unknown,0
4517,4517,2024-03-31,57,26.9,self-employed,married,tertiary,yes,-3313,yes,yes,unknown,0,unknown,0
4518,4518,2024-03-31,57,5.9,technician,married,secondary,no,295,no,no,cellular,0,unknown,0
4519,4519,2024-03-31,28,0.3,blue-collar,married,secondary,no,1137,no,no,cellular,3,other,0
